In [1]:
from secret_keys import SP_DC, SP_KEY, CLIENT_SECRET
from syrics.api import Spotify
import requests
import pandas as pd

In [2]:
import spotify_token as st

data = st.start_session(SP_DC, SP_KEY)
access_token = data[0]
expiration_date = data[1]
sp = Spotify(SP_DC)

In [3]:
def cleanLyrics(lyrics):
    lyrics = lyrics['lyrics']
    lines = lyrics['lines']
    to_return = []
    for line in lines:
        to_return.append(line['words'])
    return to_return

In [4]:
def searchArtists(name):
    endpoint = "https://api.spotify.com/v1/search"
    q = name
    type = ["artist"]

    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    params = {
        "q": q,
        "type": type
    }

    response = requests.get(endpoint, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    # Assume the first artist
    spotify_href = response.json()['artists']['items'][0]['external_urls']['spotify']
    id = spotify_href.split("/")[-1]
    return id

In [5]:
def getArtistsAlbums(id, limit=50):
    endpoint = f"https://api.spotify.com/v1/artists/{id}/albums"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    # include_groups = ["album", "single", "appears_on"]
    market = "US"
    limit = limit

    params = {
        # "include_groups": include_groups,
        "market": market,
        "limit": limit
    }

    response = requests.get(endpoint, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception("Error getting albums")
    
    albums = response.json()['items']
    to_return = []
    for album in albums:
        return_album = {
            "name": album['name'],
            "id": album['id'],
            "release_date": album['release_date'],
            "album_type": album['album_type']
        }
        to_return.append(return_album)
    return to_return

In [6]:
def getAlbumTracks(album_id):
    endpoint = f"https://api.spotify.com/v1/albums/{album_id}/tracks"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    limit = 50
    params = {
        "limit": limit
    }

    response = requests.get(endpoint, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception("Error getting tracks")
    
    tracks = response.json()['items']
    to_return = []
    for track in tracks:
        return_track = {
            "name": track['name'],
            "id": track['id'],
            "duration_ms": track['duration_ms']
        }
        to_return.append(return_track)
    return to_return

In [7]:
def generateLyrics(name, limit = 50):
    artist_id = searchArtists(name)
    albums = getArtistsAlbums(artist_id, limit)
    to_return = []
    for album in albums:
        album_name = album['name']
        album_id = album['id']
        try:
            album_tracks = getAlbumTracks(album_id)
            for track in album_tracks:
                track_id = track['id']
                if track_id == None:
                    lyrics = []
                else:
                    lyrics = cleanLyrics(sp.get_lyrics(track_id))
                    to_return.append({
                        "artist": name,
                        "album": album_name,
                        "track": track['name'],
                        "lyrics": lyrics
                    })
        except:
            pass
            
    return to_return

In [8]:
tswift = generateLyrics("Taylor Swift")
df = pd.DataFrame(tswift)
df.to_csv("tswift_lyrics.csv", index=False)

In [11]:
df

,artist,album,track,lyrics
0,Taylor Swift,1989 (Taylor's Version) [Deluxe],Welcome To New York (Taylor's Version),"[Walkin' through a crowd, the village is aglow..."
1,Taylor Swift,1989 (Taylor's Version) [Deluxe],Blank Space (Taylor's Version),"[Nice to meet you, where you been?, I could sh..."
2,Taylor Swift,1989 (Taylor's Version) [Deluxe],Style (Taylor's Version),"[Midnight, You come and pick me up, no headlig..."
3,Taylor Swift,1989 (Taylor's Version) [Deluxe],Out Of The Woods (Taylor's Version),"[Looking at it now, It all seems so simple, We..."
4,Taylor Swift,1989 (Taylor's Version) [Deluxe],All You Had To Do Was Stay (Taylor's Version),"[(Hey, hey, hey), (Hey, hey, hey), (Hey, hey, ..."
...,...,...,...,...
203,Taylor Swift,The More Red (Taylor’s Version) Chapter,I Knew You Were Trouble (Taylor's Version),"[Once upon a time, a few mistakes ago, I was i..."
204,Taylor Swift,The More Red (Taylor’s Version) Chapter,22 (Taylor's Version),[It feels like a perfect night to dress up lik...
205,Taylor Swift,Eyes Open (Taylor's Version),Eyes Open (Taylor's Version),"[Everybody's waiting, Everybody's watching, Ev..."
206,Taylor Swift,Renegade (Pop Version),Renegade - Pop Version,[I tapped on your window on your darkest night...
